In [1]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
def sentiment_scores(sentence, analyzer):
    """ create sentiment scores with the VADER analyzer
    :param sentence: sentence to create scores for
    :param analyzer: VADER sentiment analyzer
    :return score: a dictionary of scores (neg, neu, pos, compound)
    """
    score = analyzer.polarity_scores(sentence)
    return score

In [21]:
s = "I had a terrible day"
sentiment_scores(s, analyzer)

{'neg': 0.437, 'neu': 0.563, 'pos': 0.0, 'compound': -0.4767}

In [25]:
s = "My son had some problems at school and I feel overwhelmed."
sentiment_scores(s, analyzer)

{'neg': 0.209, 'neu': 0.698, 'pos': 0.093, 'compound': -0.3612}

In [27]:
s = "I am soooo angry at everyone!"
sentiment_scores(s, analyzer)

{'neg': 0.418, 'neu': 0.582, 'pos': 0.0, 'compound': -0.5562}

In [36]:
s = "Thank you for your responses"
sentiment_scores(s, analyzer)

{'neg': 0.0, 'neu': 0.615, 'pos': 0.385, 'compound': 0.3612}

In [35]:
s = "Thank you for your responses!"
sentiment_scores(s, analyzer)

{'neg': 0.0, 'neu': 0.589, 'pos': 0.411, 'compound': 0.4199}

In [34]:
s = "Thank you for your responses <3"
sentiment_scores(s, analyzer)

{'neg': 0.0, 'neu': 0.426, 'pos': 0.574, 'compound': 0.6597}

In [43]:
s = "Why is this so hard"
sentiment_scores(s, analyzer)

{'neg': 0.318, 'neu': 0.682, 'pos': 0.0, 'compound': -0.2183}

In [42]:
s = "Why is this so hard? I don't know how much more of this I can take"
sentiment_scores(s, analyzer)

{'neg': 0.111, 'neu': 0.889, 'pos': 0.0, 'compound': -0.2183}